In [1]:
import sys
import os
import numpy as np
from gensim.models import Word2Vec, KeyedVectors


In [2]:
proj_dir = '/Users/jujohnson/git/Hcpcs2Vec/'
sys.path.append(proj_dir)
from utils.callbacks import GensimEpochCallback  # NOQA: E402
from utils.utils import get_vocab_size, file_ts, Timer, args_to_dict  # NOQA: E402
from utils.data import load_hcpcs_corpus  # NOQA: E402


In [3]:
debug = True
ts = file_ts()

window_size = 5
min_seq_length = 2
embedding_size = 150
iters = 5
desc = f'e{embedding_size}-w{window_size}-i{iters}-t{ts}'

# I/O
data_dir = os.environ['CMS_RAW']
curr_dir = os.path.join(proj_dir, 'skipgram')
embeddings_output = os.path.join(proj_dir, 'embeddings', f'skipgram-{desc}.kv')
loss_output = os.path.join(curr_dir, 'logs', f'train-loss-{desc}.csv')
time_output = os.path.join(curr_dir, 'logs', f'train-time-{desc}.csv')



In [4]:
# load corpus
timer = Timer()
corpus = load_hcpcs_corpus(debug)
print(f'Loaded corpus with length {len(corpus)} in {timer.lap()}')


Loading sample of Medicare Part B 2012 data from /Users/jujohnson/cms-data/raw/2012/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_CY2012.csv.gz
Loaded data with shape: (1000000, 5)
Loaded data in 26.768922090530396
Generated hcpcs sequences in 45.30349898338318
Removed sequences longer than 10.0


FileNotFoundError: [Errno 2] No such file or directory: '/home/jjohn273/git/Hcpcs2Vec/data/debug-corpus.npy'

In [8]:
import pandas as pd

# 2018 Part B
data_dir = os.environ['CMS_RAW']

path = os.path.join(
    data_dir,
    '2018',
    'Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2018.csv.gz')

df_2018 = pd.read_csv(path, nrows=5)

df_2018['year'] = 2018
print(f'Loaded 2018 with shape {df_2018.shape}')

Loaded 2018 with shape (5, 27)


In [10]:
pd.set_option('display.max_columns', 150)

In [ ]:
    columns = {
        'HCPCS Code': 'hcpcs',
        'HCPCS Description': 'description',
        'Number of Services': 'count'
    }

In [11]:
df_2018.head()

,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Middle Initial of the Provider,Credentials of the Provider,Gender of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,Street Address 2 of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Country Code of the Provider,Provider Type,Medicare Participation Indicator,Place of Service,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,year
0,1,CPT copyright 2017 American Medical Associatio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502.0,MD,US,Internal Medicine,Y,F,99217.0,Hospital observation care on day of discharge,N,68.0,67.0,68.0,73.821029,381.308824,57.880294,58.212500,2018
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502.0,MD,US,Internal Medicine,Y,F,99218.0,"Hospital observation care, typically 30 minutes",N,19.0,19.0,19.0,100.883158,476.947368,76.795263,77.469474,2018
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502.0,MD,US,Internal Medicine,Y,F,99220.0,"Hospital observation care, typically 70 minutes",N,26.0,26.0,26.0,188.110769,1086.923077,147.479231,147.793462,2018
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502.0,MD,US,Internal Medicine,Y,F,99221.0,"Initial hospital inpatient care, typically 30 ...",N,24.0,24.0,24.0,102.240000,474.583333,80.155000,80.943333,2018
